In [4]:
import pandas as pd
import spacy

df_train= pd.read_csv("politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [5]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_test.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [6]:
df_train.drop('label', axis=1, inplace=True)
df_test.drop('label', axis=1, inplace=True)

In [1]:
!pip install -U spacy

In [2]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
nlp = spacy.load('es_core_news_lg')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [8]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [9]:
df_train.to_csv('train_tokenized.csv')
df_test.to_csv('test_tokenized.csv')

In [30]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_train['ideology_multiclass'] = label_encoder.fit_transform(df_train['ideology_multiclass'])
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score


# Preparar los datos
vectorizador = CountVectorizer()

# Suponiendo que df_train ya está definido y limpio
X = vectorizador.fit_transform(df_train['clean_tweet'])
y = df_train['ideology_multiclass']

# Dividir el conjunto de entrenamiento para validación (10% para test)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Definir varios modelos para probar
modelos = {
    'XGB': xgb.XGBClassifier(),
    'Regresión Logística': LogisticRegression(),
    'Naive Bayes Multinomial': MultinomialNB(),
    'SVM': SVC(),

}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_val)
    macro_f1 = f1_score(y_val, y_pred, average='macro')
    print(f"{nombre} - Macro F1 Score: {macro_f1}")


XGB - Macro F1 Score: 0.8216435621673532
Regresión Logística - Macro F1 Score: 0.9530290104928647
Naive Bayes Multinomial - Macro F1 Score: 0.8118918940823519
SVM - Macro F1 Score: 0.8467154405621897


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

# Usar CountVectorizer para la frecuencia de términos
vectorizador = CountVectorizer()

X_train_counts = vectorizador.fit_transform(df_train['clean_tweet'])
X_test_counts = vectorizador.transform(df_test['clean_tweet'])

y_train = df_train['ideology_multiclass']
y_test = df_test['ideology_multiclass']

# Definir el modelo y los parámetros para GridSearchCV
model = LogisticRegression(random_state=42, max_iter=1000)
param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2', 'l1']
}

# Crear un GridSearchCV
clf = GridSearchCV(model, param_grid, scoring='f1_macro', cv=5, verbose=1, n_jobs=-1)

# Ajustar el modelo
clf.fit(X_train_counts, y_train)

# Mejores parámetros y mejor score
print("Mejores parámetros:", clf.best_params_)
print("Mejor puntuación de cross-validation (macro F1):", clf.best_score_)

# Predecir y evaluar sobre el conjunto de prueba
y_pred = clf.predict(X_test_counts)

macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1 en el conjunto de prueba:", macro_f1)
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores parámetros: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Mejor puntuación de cross-validation (macro F1): 0.9107732100659035
Macro F1 en el conjunto de prueba: 0.6431312370487824
              precision    recall  f1-score   support

           0       0.63      0.56      0.59       117
           1       0.62      0.74      0.67       210
           2       0.66      0.66      0.66       153
           3       0.89      0.51      0.65        67

    accuracy                           0.65       547
   macro avg       0.70      0.62      0.64       547
weighted avg       0.67      0.65      0.65       547

